In [ ]:
#Imports and constants 
################################################################

import csv
import graphviz
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
from sklearn import tree
from sklearn import svm

from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, label_binarize, Imputer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from IPython.display import display, HTML

#Print options for printing the pandas dataframe in the console
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#set numpy random seed for reproducible results
np.random.seed(1234)
#to view the whole output in the jupyter console (not only an extract)
np.set_printoptions(threshold=np.nan)

#Data parameter
#################################################################
#File paths
TEST_PATH ='dataset/aps_failure_test_set.csv'
TRAIN_PATH ='dataset/aps_failure_training_set.csv'

#Columns of the readed dataframe
COLUMNS = ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009', 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000', 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001', 'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009', 'co_000', 'cp_000', 'cq_000', 'cr_000', 'cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008', 'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 'ee_000', 'ee_001', 'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009', 'ef_000', 'eg_000']
#Columns of the the final dataframe with processed histograms
HISTOGRAMMERGEDCOLUMNS = ['aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
                           'ah_000', 'ai_000',
                          'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000',
                          'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000',
                             'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 
                          'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bm_000', 
                          'bn_000', 'bo_000', 'bp_000', 'bq_000', 'br_000', 'bs_000', 'bt_000', 
                          'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 
                          'cc_000', 'cd_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 
                          'cj_000', 'ck_000', 'cl_000', 'cm_000', 
                          'co_000', 'cp_000', 'cq_000', 'cr_000',  
                          'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 
                          'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 
                          'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 
                          'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 
                          'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 
                          'ef_000', 'eg_000','ag_median','ay_median', 'az_median', 'ba_median','cn_median','cs_median','ee_median']

#possible Class Labels in the dataset
CLASS_LABELS = ['neg', 'pos']
#Parameter to skip the first [SKIPROWS] lines in the csv files before reading into the dataframe
SKIPROWS = 20

#Histogram columns
AG = np.array(['ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009'])
AY = np.array(['ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009'])
AZ = np.array(['az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009'])
BA = np.array(['ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009'])
CN = np.array(['cn_000', 'cn_001', 'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009'])
CS = np.array(['cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008', 'cs_009'])
EE = np.array(['ee_000', 'ee_001', 'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009'])

#NAN Data preprocessing parameter
NaNSTRATEGY = ['mean', 'median', 'most_frequent']

In [ ]:
#All Methods dealing with the data preprocessing 
##################################################################################

#Reads the csv file from the given path as a pandas dataframe
#returns the dataframe
def loadDatasetWithPandas(path, skiprowsNum):
    # Reading the raw data from csv file
    rawData = pd.read_csv(path, skiprows=skiprowsNum)
    # replacing the string indicating missing values with the numpy value for missing values
    NaNProcessedData = rawData.replace({'na': np.nan}, regex=True)
    return NaNProcessedData

#creates a new dataframe from the given datafame where the different histogram bins are processed into a new feature.
#returned dataframe is structured according to 'HISTOGRAMMERGEDCOLUMNS' variable
def histogramProcessing(dataf):
    data_set=dataf
    ag_ = data_set[AG]
    ay_ = data_set[AY]
    az_ = data_set[AZ]
    ba_ = data_set[BA]
    cn_ = data_set[CN]
    cs_ = data_set[CS]
    ee_ = data_set[EE]
    
    # create new dataframe for each of the above with the mean\n",
    ag_mean = ag_.mean(axis=1, skipna=True)
    ay_mean = ay_.mean(axis=1, skipna=True)
    az_mean = az_.mean(axis=1, skipna=True)
    ba_mean = ba_.mean(axis=1, skipna=True)
    cn_mean = cn_.mean(axis=1, skipna=True)
    cs_mean = cs_.mean(axis=1, skipna=True)
    ee_mean = ee_.mean(axis=1, skipna=True)

    remaining_columns = np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.setdiff1d(np.array(COLUMNS), AG), AY), AZ), BA), CN), CS), EE)
    remaining_columns= np.concatenate([['class'],remaining_columns])
    remaining_data = data_set[remaining_columns]
    return pd.concat(
        [
            remaining_data,
            ag_mean.rename('ag_median'),
            ay_mean.rename('ay_median'),
            az_mean.rename('az_median'),
            ba_mean.rename('ba_median'),
            cn_mean.rename('cn_median'),
            cs_mean.rename('cs_median'),
            ee_mean.rename('ee_median')
        ],
        axis=1
    )

#processes the NaN values in data frame according to the provided strategy (mean, median or most_frequent) usinjg scikits Imputer.
# returns a dataframe without any NaN values
def processNaNInDataset(data, strategy):
    values = data[list(COLUMNS)].values
    imp = Imputer(missing_values='NaN', strategy=strategy, axis=0)
    imp = imp.fit(values)
    cleanedValues = imp.transform(values)
    return cleanedValues

#loads the csv and uses the 3 methods above to process file (loading, Nan processing)
#returns the loaded testdatframe and trainingdataframe
def loadDataset(strategy):
    #Load Train Dataset
    trainData = loadDatasetWithPandas(TRAIN_PATH, SKIPROWS)
    #Load Test Dataset
    testData = loadDatasetWithPandas(TEST_PATH, SKIPROWS)
    #if no strategy for nan processing is provided just return the laoded dataframes
    if(strategy == None):
        return trainData, testData

    #remove class column from datset to allow conversion to float datatype
    removeClassTrain = trainData.iloc[:,1:171]
    removeClassTest = testData.iloc[:,1:171]
    # change datatype of the dataframe to allow computing of the mean of a column (otherwise an overflow will happen)
    removeClassTrain = removeClassTrain.astype('float64')
    processedTrain = processNaNInDataset(removeClassTrain,strategy)

    removeClassTest = removeClassTest.astype('float64')
    processedTest = processNaNInDataset(removeClassTest,strategy)
   

    finalTrainFrame = pd.concat(
            [
                pd.DataFrame(trainData.iloc[:,0]),
                pd.DataFrame(processedTrain, columns=COLUMNS)
            ],
            axis=1
        )

    finalTestFrame = pd.concat(
            [
                pd.DataFrame(testData.iloc[:,0]),
                pd.DataFrame(processedTest, columns=COLUMNS)
            ],
            axis=1
        )
    return finalTrainFrame, finalTestFrame

In [ ]:
#All methods dealing with evaluation
#################################################################################

#computes/plots scikit classification report, confusion matrix and calculates the overall cost of the result using the formula provided in the dataset
def evaluate(trueLabels, predictions):
    classificationRep = classification_report(trueLabels, predictions)
    print(classificationRep)
    
    confusionMatrix = confusion_matrix(trueLabels,predictions) 
    #print(confusionMatrix)
    #np.set_printoptions(precision=2)
    #Code for plotting Confusion matrix
    plt.figure()
    plot_confusion_matrix(confusionMatrix, classes=CLASS_LABELS,
                          title='Confusion matrix, without normalization')
    plt.figure()
    plot_confusion_matrix(confusionMatrix, classes=CLASS_LABELS, normalize=True,
                      title='Normalized confusion matrix')
    plt.show()
    #####################
    score = calculateOverallCostFromConfusionMatrix(confusionMatrix)
    print('Score: ' + str(score))
    return score
    
#calculates the overall cost from the given confusionMAtrix
def calculateOverallCostFromConfusionMatrix(confusionMatrix):
    #cost function from description
    score = confusionMatrix[1][0] * 500 + 10*confusionMatrix[0][1]
    return score

#plots the roc curve for given true labels and the predicted results
def plot_roc(true_lables, predictions):
    true_labels_binarized = label_binarize(true_lables, classes=['neg', 'pos']).flatten()
    predictions_binarized = label_binarize(predictions, classes=['neg', 'pos']).flatten()
    false_positives, true_positives, thresholds = roc_curve(true_labels_binarized, predictions_binarized, pos_label=1)
    roc_auc = auc(false_positives, true_positives)
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positives, true_positives, 'b',
    label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    

#taken from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#Loading of all datasets and possible imputations (thus data has to be loaded only once)
####################################################################

#Data, NAN not processed (rows with empty values removed)
trainSet, testSet = loadDataset(None)
trainSet.dropna(inplace=True)
testSet.dropna(inplace=True)
trainSet  = pd.DataFrame(trainSet)
testSet  = pd.DataFrame(testSet)

#Normal data NAN Processed (different strategys used)
meanTrainSet, meanTestSet = loadDataset(NaNSTRATEGY[0])
medianTrainSet, medianTestSet = loadDataset(NaNSTRATEGY[1])
mostFrequentTrainSet, mostFrequentTestSet = loadDataset(NaNSTRATEGY[2])

#Processed Histogram from normal, with NaN processed data
meanTrainSetHist = histogramProcessing(meanTrainSet)
meanTestSetHist = histogramProcessing(meanTestSet)
medianTrainSetHist = histogramProcessing(medianTrainSet)
medianTestSetHist = histogramProcessing(medianTestSet)
mostFrequentTrainSetHist = histogramProcessing(mostFrequentTrainSet)
mostFrequentTestSetHist = histogramProcessing(mostFrequentTestSet)

In [ ]:
#Implementation of scikit Decision Tree
###########################################################

max_depth = 100
min_samples_leaf = 10

#Use max_depth, and min_sample leafs to prevent overfitting according to http://scikit-learn.org/stable/modules/tree.html#tree 1.10.5

'''
#histogram bin as feature
classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=meanTrainSet,columns=COLUMNS), pd.DataFrame(data=meanTrainSet,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=meanTestSet,columns=COLUMNS))  
score = evaluate(pd.DataFrame(data=meanTestSet,columns=['class']), predictions)
'''

classifier = DecisionTreeClassifier(class_weight={'pos':.983, 'neg':.017},max_depth =3)  
classifier.fit(pd.DataFrame(data=mostFrequentTrainSet,columns=COLUMNS), pd.DataFrame(data=mostFrequentTrainSet,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=mostFrequentTestSet,columns=COLUMNS))  
score = evaluate(pd.DataFrame(data=mostFrequentTestSet,columns=['class']), predictions)

plot_roc(mostFrequentTestSet['class'], predictions)

'''
#Hisotgram mean as feature
classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=meanTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=meanTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=meanTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=meanTestSetHist,columns=['class']), predictions)


classifier = DecisionTreeClassifier()#class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=medianTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=medianTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=medianTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=medianTestSetHist,columns=['class']), predictions)

classifier = DecisionTreeClassifier(class_weight={'pos':.983, 'neg':.017},max_depth =max_depth, min_samples_leaf =min_samples_leaf)  
classifier.fit(pd.DataFrame(data=mostFrequentTrainSetHist,columns=HISTOGRAMMERGEDCOLUMNS), pd.DataFrame(data=mostFrequentTrainSetHist,columns=['class']))  
predictions = classifier.predict(pd.DataFrame(data=mostFrequentTestSetHist,columns=HISTOGRAMMERGEDCOLUMNS))  
score = evaluate(pd.DataFrame(data=mostFrequentTestSetHist,columns=['class']), predictions)


#Printing of the decision tree
dot_data = tree.export_graphviz(classifier, out_file=None, class_names=CLASS_LABELS, feature_names=COLUMNS, 
                         filled=True, rounded=True,) 
graph = graphviz.Source(dot_data) 
graph.render("DT")

'''

In [ ]:
#Implementation of scikit Support vector Machine(Linear)
#################################################################################
linearSvm = svm.LinearSVC(class_weight={'pos':.983, 'neg':.017})

target = pd.DataFrame(data=mostFrequentTrainSet,columns=['class'])

linearSvm.fit(pd.DataFrame(data=mostFrequentTrainSet,columns=COLUMNS),target.values.ravel())  
predictions = linearSvm.predict(pd.DataFrame(data=mostFrequentTestSet,columns=COLUMNS))  
score = evaluate(pd.DataFrame(data=mostFrequentTestSet,columns=['class']), predictions)